<a href="https://colab.research.google.com/github/orfi9/google_ml_course/blob/main/avocado_ripennes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear clasification without hidden layers\
Done in learning practice on data in which non-linear classification isn't neaded\
probability estimates made are made with sigmoid function?


## Libraries and data

In [1]:
# @title Install required libraries

!pip install google-ml-edu==0.1.2 \
    keras~=3.8.0 \
    matplotlib~=3.10.0 \
    numpy~=2.0.0 \
    pandas~=2.2.0 \
    tensorflow~=2.18.0

print('\n\nAll requirements successfully installed.')



All requirements successfully installed.


In [2]:
# @title Load the imports

import io
import keras
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import ml_edu.experiment
import ml_edu.results
import numpy as np
import pandas as pd
import plotly.express as px

# The following lines adjust the granularity of reporting.
# pd.options.display.max_rows = 10
# pd.options.display.float_format = "{:.1f}".format

print("Ran the import statements.")

Ran the import statements.


In [3]:
# @title Get data from kaggle
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amldvvs/avocado-ripeness-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/avocado-ripeness-classification-dataset


## Load, inspect, normalizate, get dummies

In [4]:
df = pd.read_csv(path+'/avocado_ripeness_dataset.csv')
df.head()

,firmness,hue,saturation,brightness,color_category,sound_db,weight_g,size_cm3,ripeness
0,14.5,19,40,26,black,34,175,261,ripe
1,71.7,53,69,75,green,69,206,185,pre-conditioned
2,88.5,60,94,46,dark green,79,220,143,hard
3,93.8,105,87,41,dark green,75,299,140,hard
4,42.5,303,58,32,purple,63,200,227,breaking


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   firmness        250 non-null    float64
 1   hue             250 non-null    int64  
 2   saturation      250 non-null    int64  
 3   brightness      250 non-null    int64  
 4   color_category  250 non-null    object 
 5   sound_db        250 non-null    int64  
 6   weight_g        250 non-null    int64  
 7   size_cm3        250 non-null    int64  
 8   ripeness        250 non-null    object 
dtypes: float64(1), int64(6), object(2)
memory usage: 17.7+ KB


In [6]:
#@title feature correlation
df.corr(numeric_only=True)

,firmness,hue,saturation,brightness,sound_db,weight_g,size_cm3
firmness,1.000000,0.038071,0.807693,0.712119,0.951817,0.743636,-0.943758
hue,0.038071,1.000000,0.204663,0.172986,0.150709,0.077141,-0.035083
saturation,0.807693,0.204663,1.000000,0.681025,0.803399,0.631171,-0.808331
brightness,0.712119,0.172986,0.681025,1.000000,0.680530,0.557013,-0.707422
sound_db,0.951817,0.150709,0.803399,0.680530,1.000000,0.735709,-0.925431
weight_g,0.743636,0.077141,0.631171,0.557013,0.735709,1.000000,-0.722762
size_cm3,-0.943758,-0.035083,-0.808331,-0.707422,-0.925431,-0.722762,1.000000


In [7]:
#@title ripeness are uniformly distributet
df.ripeness.value_counts()

,count
ripeness,
ripe,50
pre-conditioned,50
hard,50
breaking,50
firm-ripe,50


In [8]:
#@title feature z-normalization
features_mean = df.mean(numeric_only=True)
features_std = df.std(numeric_only=True)

numeric_features = df.select_dtypes(include=np.number, exclude=object)
normalized_df = (numeric_features - features_mean) / features_std

normalized_df['ripeness'] = df.ripeness
normalized_df['color_category'] = df.color_category

In [9]:
# @title one hot encoding
norm_enc_df = pd.get_dummies(normalized_df, columns=['color_category'], drop_first=True, dtype=float)
norm_enc_df = pd.get_dummies(norm_enc_df, columns=['ripeness'], drop_first=False, dtype=float)
norm_enc_df.head()

,firmness,hue,saturation,brightness,sound_db,weight_g,size_cm3,color_category_dark green,color_category_green,color_category_purple,ripeness_breaking,ripeness_firm-ripe,ripeness_hard,ripeness_pre-conditioned,ripeness_ripe
0,-1.324101,-0.912977,-1.401611,-0.997683,-1.735495,-1.313405,0.935922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.772852,-0.622720,0.267248,1.565576,0.793749,-0.412379,-0.422663,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.388740,-0.562962,1.705919,0.048545,1.516390,-0.005464,-1.173460,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.583038,-0.178798,1.303091,-0.213012,1.227334,2.290699,-1.227088,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.297621,1.511521,-0.365768,-0.683815,0.360164,-0.586771,0.328134,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## Overview
Classes are pretty much can be linearly split. \
So at first we might dont need non-linear learning NN with hidden layers.
Let's see if we can use cross-entrophy loss with sigmoid as logistic regression output of probabilities of belonging to the corresponding classes.

In [10]:
#@title scatterplot of all data
px.scatter_3d(
    normalized_df,
    x='firmness',
    y='saturation',
    z='sound_db',
    color='ripeness',
).show()

In [11]:
# Name of the features we'll train our model on.
input_features = [
    'firmness',
    'saturation',
    'sound_db',
]

In [12]:
# @title Split on train, test, validation datasets 0.8 0.1 0.1
# Create indices at the 80th and 90th percentiles
number_samples = len(norm_enc_df)
index_80th = round(number_samples * 0.8)
index_90th = index_80th + round(number_samples * 0.1)

# Randomize order and split into train, validation, and test with a .8, .1, .1 split
shuffled_dataset = norm_enc_df.sample(frac=1, random_state=100)
train_data = shuffled_dataset.iloc[0:index_80th]
validation_data = shuffled_dataset.iloc[index_80th:index_90th]
test_data = shuffled_dataset.iloc[index_90th:]

print(
f'train data: {train_data.shape} \n\
test data: {test_data.shape} \n\
validation dataset: {validation_data.shape}'
)

# Show the first five rows of the last split
print('\ntest data')
test_data.head()

train data: (200, 15) 
test data: (25, 15) 
validation dataset: (25, 15)

test data


,firmness,hue,saturation,brightness,sound_db,weight_g,size_cm3,color_category_dark green,color_category_green,color_category_purple,ripeness_breaking,ripeness_firm-ripe,ripeness_hard,ripeness_pre-conditioned,ripeness_ripe
93,1.451062,-0.289779,0.785169,1.147084,1.155070,2.290699,-1.441601,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
137,-0.565239,1.289560,0.382341,0.467036,-0.940590,0.168928,1.096807,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
144,0.200956,1.673723,-0.595955,-0.317635,0.360164,-0.703033,-0.154521,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
58,-0.572571,-1.049568,-1.113877,-1.677731,-0.290213,-0.703033,0.328134,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
60,-0.297621,1.340781,0.670076,0.414725,0.360164,-1.109948,0.310258,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## Model training and predictions

In [13]:
#@title split all datasets by features and labels datasets
label_columns = train_data.columns[-5:].to_list() # ather one-hot encoding last 5 columns are label columns

train_features = train_data.drop(columns=label_columns)
train_labels = train_data[label_columns].to_numpy()
validation_features = validation_data.drop(columns=label_columns)
validation_labels = validation_data[label_columns].to_numpy()
test_features = test_data.drop(columns=label_columns)
test_labels = test_data[label_columns].to_numpy()

In [14]:
keras.utils.set_random_seed(42)

In [15]:
#@title Model without hidden layers
"""
essentialy a linear model, similar to logistic regression\
model that does not learn coplexity of data through non-linearity\
only sigmoid function is used for making a prediction
"""

linear_model = keras.Sequential([
    keras.layers.Input(shape=(len(input_features),)),
    keras.layers.Dense(len(label_columns), activation='sigmoid')
])

metrics = [
    # "accuracy"
    keras.metrics.CategoricalAccuracy(
        name="categorical_accuracy",
        dtype=None
    ),

]
# Compile the model
linear_model.compile(optimizer=keras.optimizers.RMSprop(0.01),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=metrics)
linear_model.fit(train_features[input_features], train_labels, epochs=40, batch_size=20)



Epoch 1/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - categorical_accuracy: 0.3610 - loss: 1.5263
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - categorical_accuracy: 0.4419 - loss: 1.3283 
Epoch 3/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.4071 - loss: 1.2239 
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - categorical_accuracy: 0.4023 - loss: 1.1462
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - categorical_accuracy: 0.4393 - loss: 1.0842
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - categorical_accuracy: 0.4865 - loss: 1.0329  
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.5128 - loss: 0.9893 
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.5240 - loss: 0.9514 
Epoch 9/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.5370 - loss: 0.9177 
Epoch 10/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - categorical_accuracy: 0.5895 - loss: 0.8873
Epoch 11/40
10/10 ━━━━━━━━━━

In [16]:
for layer in linear_model.layers:
  print(f'weights: {layer.get_weights()[0]}, \nbiases: {layer.get_weights()[1]}')
  print('----------------------')

weights: [[-0.76638854 -2.2075481   2.0507193   1.8207554  -2.6076405 ]
 [ 0.17520492 -0.27990752  1.305006    0.55082893 -0.7509559 ]
 [ 0.84993124 -0.7796711   2.840222    0.5740479  -3.1279953 ]], 
biases: [ 1.5628775   0.732968   -1.9451984   0.74765563 -2.1476986 ]
----------------------


In [17]:
#@title model with 1 hidden layer of 5 nodes
model_hidden_layer = keras.Sequential([
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(len(label_columns), activation='softmax')
])

metrics = [
    # "accuracy"
    keras.metrics.CategoricalAccuracy(
        name="categorical_accuracy",
        dtype=None
    ),

]
# Compile the model
model_hidden_layer.compile(optimizer=keras.optimizers.RMSprop(0.01),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=metrics)
model_hidden_layer.fit(train_features[input_features], train_labels, epochs=40, batch_size=20)

Epoch 1/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - categorical_accuracy: 0.2643 - loss: 1.6061
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.2528 - loss: 1.4353 
Epoch 3/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.3633 - loss: 1.2672
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - categorical_accuracy: 0.3942 - loss: 1.1347 
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - categorical_accuracy: 0.4439 - loss: 1.0349 
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - categorical_accuracy: 0.4878 - loss: 0.9556 
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.5375 - loss: 0.8873 
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.5773 - loss: 0.8234 
Epoch 9/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.6411 - loss: 0.7664 
Epoch 10/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.6888 - loss: 0.7126 
Epoch 11/40
10/10 ━━━━━━━━━━━━

In [18]:
for layer in model_hidden_layer.layers:
  print(f'weights: {layer.get_weights()[0]}, \nbiases: {layer.get_weights()[1]}')
  print('----------------------')

weights: [[-1.0425861   1.1670439   1.9662389   2.2700584   0.18428333]
 [-1.3838334  -0.18048176  0.09636509  0.38905492 -0.21564464]
 [-2.6377094   1.6384778   0.92042696  2.1990328   0.6880087 ]], 
biases: [ 1.3579817   1.6070126  -0.08599856 -1.133217    2.0067654 ]
----------------------
weights: [[-1.1774161  -0.09185458 -2.2038205  -3.7215858   1.1782255 ]
 [ 1.2746607  -3.3278615  -0.01445907 -0.2845594  -3.5198379 ]
 [-2.289265   -2.407186    0.74993294  0.8073866  -1.0268682 ]
 [-3.0738554  -1.4046543   1.6001629  -0.538875   -0.91947913]
 [ 0.86056286  1.4905987  -1.6013803   0.49815974 -2.3930774 ]], 
biases: [ 0.9606136   1.6046994  -2.0301516   0.90132314 -1.4066124 ]
----------------------


## Predictions

In [19]:
test_data = test_data.reset_index(drop=True)

In [20]:
# reverse one-hot encoding
test_data['ripeness'] = test_data.loc[:, label_columns].apply(lambda x: label_columns[np.argmax(x)][9:], axis=1)

In [21]:
predictions_of_model_without_hidden_layer = linear_model.predict(test_data[['firmness','saturation', 'sound_db']])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [22]:
np.round(predictions_of_model_without_hidden_layer, 3)[15:20]

array([[0.798, 0.994, 0.   , 0.05 , 0.998],
       [0.855, 0.207, 0.878, 0.928, 0.002],
       [0.828, 0.684, 0.119, 0.67 , 0.11 ],
       [0.829, 0.981, 0.001, 0.11 , 0.976],
       [0.781, 0.996, 0.   , 0.026, 0.999]], dtype=float32)

In [23]:
test_data.loc[:, 'predictions_of_model_without_hidden_layer'] = [f'pred_{label_columns[np.argmax(row)][9:]}' for row in predictions_of_model_without_hidden_layer]

In [24]:
px.scatter_3d(
    test_data,
    x='firmness',
    y='saturation',
    z='sound_db',
    color='ripeness',
    symbol = 'predictions_of_model_without_hidden_layer',
    title = 'predictions_of_model_without_hidden_layer'
).show()

In [25]:
# with one hidden layer
predictions_of_model_with_hidden_layer = model_hidden_layer.predict(test_data[['firmness','saturation', 'sound_db']])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


In [26]:
np.round(predictions_of_model_with_hidden_layer, 3)[15:20]

array([[0.   , 0.067, 0.   , 0.   , 0.933],
       [0.009, 0.   , 0.046, 0.944, 0.   ],
       [0.979, 0.02 , 0.   , 0.001, 0.   ],
       [0.   , 0.8  , 0.   , 0.   , 0.2  ],
       [0.   , 0.034, 0.   , 0.   , 0.966]], dtype=float32)

In [27]:
test_data['predictions_of_model_with_hidden_layer'] = [f'pred_{label_columns[np.argmax(row)][9:]}' for row in predictions_of_model_with_hidden_layer]

In [28]:
px.scatter_3d(
    test_data,
    x='firmness',
    y='saturation',
    z='sound_db',
    color='ripeness',
    symbol = 'predictions_of_model_with_hidden_layer',
    title = 'predictions_of_model_with_hidden_layer'
).show()